## Introduction to `robotoc` 
# 4: Optimal Control of a Quadruped

## The goal of this chapter
Here, we learn how to construct and solve an optimal control problem (OCP) for a quadruped ANYmal.

First, construct `robotoc.Robot` of a quadruped robot ANYmal.

In [ ]:
import robotoc
contact_frames = ['LF_FOOT', 'LH_FOOT', 'RF_FOOT', 'RH_FOOT']
contact_types = [robotoc.ContactType.PointContact for _ in contact_frames]
baumgarte_time_step = 0.05
path_to_urdf = 'urdf/anymal_b_simple_description/urdf/anymal.urdf'
anymal = robotoc.Robot(path_to_urdf, robotoc.BaseJointType.FloatingBase, 
                       contact_frames, contact_types, baumgarte_time_step)

## Cost Function
Next, we create the cost function of the optimal control problem (OCP).   
In `robotoc`, we create the cost function `robotoc.CostFunction` (`std::shared_ptr<robotoc::CostFunction>` in C++) by combining various **cost components**.  
That is, `robotoc.CostFunction` (`std::shared_ptr<robotoc::CostFunction>` in C++) is a collection of the **cost components** .   
Each **cost component** inherits `robotoc.CostFunctionComponentBase` (`robotoc::CostFunctionComponentBase` in C++).    
By doing so, we can easily combine many kinds of **cost components**.

Basic and useful **cost components** are already implemented in `robotoc` .  
In this example, we utilize the **cost component on the configuration (i.e., cost on the joint-space)**, **cost components on the end-effector position (i.e., cost on the task-space position)**, and **cost component on the center of mass (CoM) position**.

### Gait Pattern
Before constructing the cost function, we define parameters for the gait pattern.   
Here, we consider a simple trot.

In [ ]:
import numpy as np
step_length = np.array([0.15, 0, 0])
step_height = 0.1
swing_time = 0.5
double_support_time = 0.04
swing_start_time = 0.04

### Configuration Space Cost
First, we define a configuration-space cost component.
We set the reference values and weight parameters of the configuration space cost component. 
Naming conventions are as follows:
We set the reference values and weight parameters of the configuration space cost compoonent.  
Naming conventions are as follows:
- `q_ref`: reference configuration (joint positions)
- `q_weight`: weight on the configuration 
- `v_weight`: weight on the generalized velocity (joint velocities)
- `a_weight`: weight on the generalized acceleration (joint accelerations)
- `u_weight`: weight on the joint torques
- `q_weight_terminal`, `v_weight_terminal`: weights on the configuration and generalized velocity at the terminal time of the horizon
- `q_weight_impulse`, `v_weight_impulse`: weights on the configuration and generalized velocity at the impulse instant (just before the impact, i.e., touch-down of the feet onto the ground)
- `dv_weight_impulse`: weight on the impulse change in the generalized velocity at the impulse stage due to the impact

In [ ]:
q_standing = np.array([0, 0, 0.4792, 0, 0, 0, 1, 
                       -0.1,  0.7, -1.0, 
                       -0.1, -0.7,  1.0, 
                        0.1,  0.7, -1.0, 
                        0.1, -0.7,  1.0])
q_weight = np.array([0, 0, 0, 250000, 250000, 250000, 
                     0.0001, 0.0001, 0.0001, 
                     0.0001, 0.0001, 0.0001,
                     0.0001, 0.0001, 0.0001,
                     0.0001, 0.0001, 0.0001])
v_weight = np.array([100, 100, 100, 100, 100, 100, 
                     1, 1, 1, 
                     1, 1, 1,
                     1, 1, 1,
                     1, 1, 1])
u_weight = np.full(anymal.dimu(), 1.0e-01)
q_weight_terminal = q_weight
v_weight_terminal = v_weight
q_weight_impulse = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 
                             100, 100, 100, 
                             100, 100, 100,
                             100, 100, 100,
                             100, 100, 100])
v_weight_impulse = np.full(anymal.dimv(), 100)
config_cost = robotoc.ConfigurationSpaceCost(anymal)
config_cost.set_q_ref(q_standing)
config_cost.set_q_weight(q_weight)
config_cost.set_v_weight(v_weight)
config_cost.set_u_weight(u_weight)
config_cost.set_q_weight_terminal(q_weight_terminal)
config_cost.set_v_weight_terminal(v_weight_terminal)
config_cost.set_q_weight_impulse(q_weight_impulse)
config_cost.set_v_weight_impulse(v_weight_impulse)

### Task Space Cost
Second, we define task-space cost components.  
We set the reference positions of the end-effector (each foot) and weight parameters.  
This kind of costs can be constructed as `robotoc.TaskSpace3DCost` (`std::shared_ptr<robotoc::TaskSpace3DCost>` in C++).  
In each `robotoc.TaskSpace3DCost`, we specify a periodic position reference that inherits `robotoc.TaskSpace3DRefBase` (`robotoc::TaskSpace3DRefBase` in C++).  
For robot walking, we provide `robotoc.PeriodicSwingFootRef` (`robotoc::PeriodicSwingFootRef` in C++), in which we specify the initial position, step length, step height, swing-start time, swing time, support time, and whether the initial step-length is half or not.

To specify the initial positions of all feet, we first obtain the feet' positions at the standing pose.

In [ ]:
anymal.forward_kinematics(q_standing)
x3d0_LF = anymal.frame_position('LF_FOOT')
x3d0_LH = anymal.frame_position('LH_FOOT')
x3d0_RF = anymal.frame_position('RF_FOOT')
x3d0_RH = anymal.frame_position('RH_FOOT')

We then construct the feet refs as

In [ ]:
LF_t0 = swing_start_time + swing_time + double_support_time
LH_t0 = swing_start_time
RF_t0 = swing_start_time
RH_t0 = swing_start_time + swing_time + double_support_time
LF_foot_ref = robotoc.PeriodicSwingFootRef(x3d0_LF, step_length, step_height, 
                                           LF_t0, swing_time, 
                                           swing_time+2*double_support_time, False)
LH_foot_ref = robotoc.PeriodicSwingFootRef(x3d0_LH, step_length, step_height, 
                                           LH_t0, swing_time, 
                                           swing_time+2*double_support_time, True)
RF_foot_ref = robotoc.PeriodicSwingFootRef(x3d0_RF, step_length, step_height, 
                                           RF_t0, swing_time, 
                                           swing_time+2*double_support_time, True)
RH_foot_ref = robotoc.PeriodicSwingFootRef(x3d0_RH, step_length, step_height, 
                                           RH_t0, swing_time, 
                                           swing_time+2*double_support_time, False)

Now we can construct time-varying task space cost components of feet as

In [ ]:
LF_cost = robotoc.TaskSpace3DCost(anymal, 'LF_FOOT', LF_foot_ref)
LH_cost = robotoc.TaskSpace3DCost(anymal, 'LH_FOOT', LH_foot_ref)
RF_cost = robotoc.TaskSpace3DCost(anymal, 'RF_FOOT', RF_foot_ref)
RH_cost = robotoc.TaskSpace3DCost(anymal, 'RH_FOOT', RH_foot_ref)
swing_foot_weight = np.full(3, 1.0e06)
LF_cost.set_weight(swing_foot_weight)
LH_cost.set_weight(swing_foot_weight)
RF_cost.set_weight(swing_foot_weight)
RH_cost.set_weight(swing_foot_weight)

### Center of Mass (CoM) Cost

We also define a cost component on the position of center of mass (CoM).   
The way to define this cost is similar to the above.   
First, we create the time-varying reference of the CoM position as 

In [ ]:
com_ref0 = anymal.com()
vcom_ref = 0.5 * step_length / swing_time
com_ref = robotoc.PeriodicCoMRef(com_ref0, vcom_ref, swing_start_time, swing_time, 
                                 double_support_time, True)

With this reference, we can create the time-varying CoM cost as 

In [ ]:
com_cost = robotoc.CoMCost(anymal, com_ref)
com_cost.set_weight(np.full(3, 1.0e06))

We then make the **cost function** `robotoc.CostFunction` and add the **configuration cost component**, **task-space cost components**, and **CoM cost components** to it.

In [ ]:
cost = robotoc.CostFunction()
cost.push_back(config_cost)
cost.push_back(LF_cost)
cost.push_back(LH_cost)
cost.push_back(RF_cost)
cost.push_back(RH_cost)
cost.push_back(com_cost)

## Constraints 
We can construct constraints in the same way as the cost function.  
In `robotoc`, we create the constraints `robotoc.Constraints` (`std::shared_ptr<robotoc::Constraints>` in C++) by combining various **constraint components**.  
That is, the `robotoc.Constraints` (`std::shared_ptr<robotoc::Constraints>` in C++) is a collection of the **constraint components** .  
Each **constraints component** inherits `robotoc.ConstraintComponentBase` (`robotoc::ConstraintComponentBase` in C++).   
By doing so, we can easily combine many kinds of **constraint components**.

Basic and useful **constraint components** are already implemented in robotoc.  
In this example, we utilize **constraint components on the joint position, velocity, and torque limits**.

In [ ]:
joint_position_lower  = robotoc.JointPositionLowerLimit(anymal)
joint_position_upper  = robotoc.JointPositionUpperLimit(anymal)
joint_velocity_lower  = robotoc.JointVelocityLowerLimit(anymal)
joint_velocity_upper  = robotoc.JointVelocityUpperLimit(anymal)
joint_torques_lower   = robotoc.JointTorquesLowerLimit(anymal)
joint_torques_upper   = robotoc.JointTorquesUpperLimit(anymal)

We also use the **friction cone constraint component** in this example.  
We specify the friction coefficient, which is denoted as `mu`.

In [ ]:
mu = 0.7
friction_cone = robotoc.FrictionCone(anymal, mu)

We then make the **constraints** `robotoc.Constraints` and add these **constraint components**.

In [ ]:
constraints = robotoc.Constraints()
constraints.push_back(joint_position_lower)
constraints.push_back(joint_position_upper)
constraints.push_back(joint_velocity_lower)
constraints.push_back(joint_velocity_upper)
constraints.push_back(joint_torques_lower)
constraints.push_back(joint_torques_upper)
constraints.push_back(friction_cone)

## Contact Sequence
The **contact sequence** (`robotoc.ContactSequence`) describes the sequence of the **contact status** (`robotoc.ContactStatus`), which containts contact information such as which contacts are active at each time and contact placements (i.e., positions and rotations of the contacts).   
We pass the `robotoc.ContactSequence` to the OCP solver.   
Then the OCP solver automatically constructs an OCP in which the robot dynamics follow the input contact sequence.

In this example, we first specify the number of cycles of the trotting gait.

In [ ]:
cycle = 3

In constructing the contact sequence, we have to specify the maximum possible number of the discrete events (i.e., touch-down and lift-off in this example).   
We construct the contact sequence as 

In [ ]:
max_num_each_discrete_events = 2*cycle
contact_sequence = robotoc.ContactSequence(anymal, max_num_each_discrete_events)

We sequentially build the contact sequence of the biped walking.  
First, we make `robotoc.ContactStatus` object.   
We set which contacts are active and contact placements in this object.  
At the beginning, the robot is standing whose configuration is given as `q_standing` .  
This is specified as follows.

In [ ]:
anymal.forward_kinematics(q_standing)
x3d0_LF = anymal.frame_position('LF_FOOT')
x3d0_LH = anymal.frame_position('LH_FOOT')
x3d0_RF = anymal.frame_position('RF_FOOT')
x3d0_RH = anymal.frame_position('RH_FOOT')
contact_positions = {'LF_FOOT': x3d0_LF, 'LH_FOOT': x3d0_LH, 'RF_FOOT': x3d0_RF, 'RH_FOOT': x3d0_RH} 
contact_status_standing = anymal.create_contact_status()
contact_status_standing.activate_contacts(['LF_FOOT', 'LH_FOOT', 'RF_FOOT', 'RH_FOOT'])
contact_status_standing.set_contact_placements(contact_positions)

We initialize the contact sequence with this contact status.

In [ ]:
contact_sequence.init_contact_sequence(contact_status_standing)

Next, we add the contact sequence in which the contacts in `LF_FOOT` and `RH_FOOT` are active and these in `LH_FOOT` and `RF_FOOT` are inactive (that is, `LH_FOOT` and `RF_FOOT` are swinging).    
This is expressed as follows.

In [ ]:
contact_status_lhrf_swing = anymal.create_contact_status()
contact_status_lhrf_swing.activate_contacts(['LF_FOOT', 'RH_FOOT'])
contact_status_lhrf_swing.set_contact_placements(contact_positions)

We add (push_back) this contact status into the contact sequence.

In [ ]:
contact_sequence.push_back(contact_status_lhrf_swing, swing_start_time)

In the next phase, the four feet have active contacts.  
Now the positions of `LH_FOOT` and `RF_FOOT` went forward 0.5 times `step_length` (because length of the initial step is half of the other steps).  
This is described as follows.

In [ ]:
contact_positions['LH_FOOT'] += 0.5 * step_length
contact_positions['RF_FOOT'] += 0.5 * step_length
contact_status_standing.set_contact_placements(contact_positions)
contact_sequence.push_back(contact_status_standing, swing_start_time+swing_time)

Next, `LF_FOOT` and `RH_FOOT` are swinging, i.e., the contacts in `LF_FOOT` and `RH_FOOT` are inactive and these of `LH_FOOT` and `RF_FOOT` are active.  
This is written as 

In [ ]:
contact_status_lfrh_swing = anymal.create_contact_status()
contact_status_lfrh_swing.activate_contacts(['LH_FOOT', 'RF_FOOT'])
contact_status_lfrh_swing.set_contact_placements(contact_positions)
contact_sequence.push_back(contact_status_lfrh_swing, 
                           swing_start_time+swing_time+double_support_time)

Next, the four feet become having contacts again.   
`LF_FOOT` and `RH_FOOT` went forward with `step_length`.

In [ ]:
contact_positions['LF_FOOT'] += step_length
contact_positions['RH_FOOT'] += step_length
contact_status_standing.set_contact_placements(contact_positions)
contact_sequence.push_back(contact_status_standing, 
                           swing_start_time+2*swing_time+double_support_time)

This is repeated until the specified number of cycles.

In [ ]:
for i in range(cycle-1):
    t1 = swing_start_time + (i+1)*(2*swing_time+2*double_support_time)
    contact_status_lhrf_swing.set_contact_placements(contact_positions)
    contact_sequence.push_back(contact_status_lhrf_swing, t1)

    contact_positions['LH_FOOT'] += step_length
    contact_positions['RF_FOOT'] += step_length
    contact_status_standing.set_contact_placements(contact_positions)
    contact_sequence.push_back(contact_status_standing, t1+swing_time)

    contact_status_lfrh_swing.set_contact_placements(contact_positions)
    contact_sequence.push_back(contact_status_lfrh_swing, 
                               t1+swing_time+double_support_time)

    contact_positions['LF_FOOT'] += step_length
    contact_positions['RH_FOOT'] += step_length
    contact_status_standing.set_contact_placements(contact_positions)
    contact_sequence.push_back(contact_status_standing, 
                               t1+2*swing_time+double_support_time)

## Optimal Control Problem (OCP) and Solver

Now we can construct the optimal control problem (OCP) and solvers.  
First, set the length of the horizon `T` and the number of discretization grids `N` as 

In [ ]:
T = swing_start_time + cycle*(2*double_support_time+2*swing_time)
dt = 0.02
N = int(np.floor(T/dt))    

Since this robot involves contacts and a floating base, we construct `robotoc.OCP` (`robotoc::OCP` in C++).      
Here, we again specify the maximum possible number of the discrete events.

In [ ]:
ocp = robotoc.OCP(robot=anymal, cost=cost, constraints=constraints, 
                  T=T, N=N, max_num_each_discrete_events=max_num_each_discrete_events)

Then makes the OCP solver `robotoc.OCPSolver` (`robotoc::OCPSolver` in C++) for this OCP.  
This solver has the following characteristics: 
- Direct multiple-shooting method and primal-dual interior point method.
- Lifted-contact dynamics for robotic systems with contacts.
- Constraint-transformation for pure-state equality constraints.
- Riccati recursion to compute the Newton steps.
- Switching time optimization via Riccati recursion (we do not use this function in this example).

We can specify some solver options by `robotoc.SolverOptions` (we use the default settings in the following).  
We also specify the number of threads used in parallel computing in the direct multiple-shooting method.

In [ ]:
solver_options = robotoc.SolverOptions()
ocp_solver = robotoc.OCPSolver(ocp=ocp, contact_sequence=contact_sequence, 
                               solver_options=solver_options, nthreads=4)

## Solve the OCP

We solve the OCP in which the initial state (`q`, `v`) and initial time (`t`) are given as follows.

In [ ]:
t = 0.
q = q_standing.copy()
v = np.zeros(anymal.dimv())

We initialize the OCP solver.  
The following appropriate initialization is very simple but important to solve the OCP efficiently.   
First, we set the initial guess of the solution as the above `q` and `v`.   
Second, we set the initial guess of the solution of contact force `f` by the total weight of the robot.

In [ ]:
ocp_solver.set_solution("q", q)
ocp_solver.set_solution("v", v)
f_init = np.array([0.0, 0.0, 0.25*anymal.total_weight()])
ocp_solver.set_solution("f", f_init)

We also initialize the solver to treat the inequality constraints.   
The following methods do it. (Specifically, it initializes slack and dual variables of the primal-dual interior point method).

In [ ]:
ocp_solver.init_constraints(t)

To see how the current iterate is close to the stationary points, we see the **KKT error**, which is an l2-norm of the Karush–Kuhn–Tucker (KKT) conditions (first-order necessary conditions of the optimality).   
After the above initialization, the KKT error is 

In [ ]:
ocp_solver.KKT_error(t, q, v)

Now we solve the OCP

In [ ]:
ocp_solver.solve(t, q, v)

We can see that the KKT error is very small. (Specifically, smaller than the threshold set in `robotoc.SolverOption`).

In [ ]:
ocp_solver.KKT_error(t, q, v)

We can also see the stats of the solver as follows.

In [ ]:
stats = ocp_solver.get_solver_statistics()
print(stats)

We plot the joint position, velocities, and torques as 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn

plt.rcParams['figure.figsize'] = 10, 9
fig = plt.figure()
ax_q = fig.add_subplot(3, 1, 1)
ax_v = fig.add_subplot(3, 1, 2)
ax_u = fig.add_subplot(3, 1, 3)

# Plot results
ax_q.plot(np.linspace(t, t+T, N+1), ocp_solver.get_solution('q'))
ax_v.plot(np.linspace(t, t+T, N+1), ocp_solver.get_solution('v'))
ax_u.plot(np.linspace(t, t+T-(T/N), N), ocp_solver.get_solution('u'))

ax_q.set_xlabel('')
ax_v.set_xlabel('')
ax_u.set_xlabel('t [s]')

ax_q.set_ylabel('q [rad]')
ax_v.set_ylabel('v [rad/s]')
ax_u.set_ylabel('u [Nm]')

`robotoc` provides a module to plot the contact forces with friction cone constraints.  
In the plot, the hatched region shows the infeasible region of fx and fy.

In [ ]:
plot_f = robotoc.utils.PlotContactForce(mu=mu)
plot_f.figsize = 14, 8
plot_f.wspace = 0.2
plot_f.hspace = 0.2
plot_f.legend_bbox_to_anchor = (-0.2, 2.7)
plot_f.plot(f_data=ocp_solver.get_solution('f', 'WORLD'), 
            t=ocp_solver.get_time_discretization().time_points())

`robotoc` also provides a module to plot the convergence.

In [ ]:
plot_kkt = robotoc.utils.PlotConvergence()
#plot_kkt.ylim = [0., 1.5]
plot_kkt.figsize = 6, 4
plot_kkt.plot(kkt_data=ocp_solver.get_solver_statistics().kkt_error)

We can also visualize the solution trajectory.  
If we choose `'gepetto'` as the `viewer_type`, we can also see the contact forces and friction cones.

In [ ]:
viewer = robotoc.utils.TrajectoryViewer(path_to_urdf=path_to_urdf, 
                                        base_joint_type=robotoc.BaseJointType.FloatingBase,
                                        viewer_type='gepetto')
viewer.set_contact_info(anymal.contact_frames(), mu)
discretization = ocp_solver.get_time_discretization()
viewer.display(discretization.time_steps(), ocp_solver.get_solution('q'), 
               ocp_solver.get_solution('f', 'WORLD'))